In [1]:
import ibis
from pyspark.sql import SparkSession
import spark_df_profiling

In [2]:

session = SparkSession.builder.getOrCreate()
ipsc = ibis.pyspark.connect(session)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/26 22:39:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# df_rdd = session.read.csv("../data/sample_big_mortgage_file.csv.gz", header=True, sep="|")

df_rdd = session.read.csv("../data/2020_lar.txt.gz", header=True, sep="|")

In [4]:
# Try making dataframe a table and query with SQL

df_rdd.createTempView("tmp_df_table")

22/09/26 22:39:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
ipsc

In [6]:
ipsc.list_databases()

['default']

In [7]:
ipsc.list_tables()


['tmp_df_table']

In [8]:
my_table = ipsc.table("tmp_df_table")

In [9]:
for col in my_table.columns:
    if col[0] == "d":
        print(col)

derived_msa_md
derived_loan_product_type
derived_dwelling_category
derived_ethnicity
derived_race
derived_sex
discount_points
debt_to_income_ratio
denial_reason_1
denial_reason_2
denial_reason_3
denial_reason_4


In [10]:
query = "SELECT * FROM tmp_df_table LIMIT 5"

In [11]:
query = "SELECT DISTINCT derived_sex FROM tmp_df_table"

In [12]:
df1 = session.sql(query)

In [13]:
df1.show()

+-----------------+
|      derived_sex|
+-----------------+
|Sex Not Available|
|           Female|
|            Joint|
|             Male|
+-----------------+



In [14]:
# df_rdd.count()

In [15]:
### Now the Ibis approach

ibis.options.interactive = True

In [16]:
ipsc.current_database

'default'

In [17]:
ipsc.exists_table("tmp_df_table")

/var/folders/wc/zhl433ms1fx5rvtvcrkw6dgw0000gn/T/ipykernel_6545/3364586496.py:1: FutureWarning: `exists_table` is deprecated as of v2.0; use `name in client.list_tables()`
  ipsc.exists_table("tmp_df_table")


True

In [18]:
ipsc.table("tmp_df_table").columns

['activity_year',
 'lei',
 'derived_msa_md',
 'state_code',
 'county_code',
 'census_tract',
 'conforming_loan_limit',
 'derived_loan_product_type',
 'derived_dwelling_category',
 'derived_ethnicity',
 'derived_race',
 'derived_sex',
 'action_taken',
 'purchaser_type',
 'preapproval',
 'loan_type',
 'loan_purpose',
 'lien_status',
 'reverse_mortgage',
 'open_end_line_of_credit',
 'business_or_commercial_purpose',
 'loan_amount',
 'combined_loan_to_value_ratio',
 'interest_rate',
 'rate_spread',
 'hoepa_status',
 'total_loan_costs',
 'total_points_and_fees',
 'origination_charges',
 'discount_points',
 'lender_credits',
 'loan_term',
 'prepayment_penalty_term',
 'intro_rate_period',
 'negative_amortization',
 'interest_only_payment',
 'balloon_payment',
 'other_nonamortizing_features',
 'property_value',
 'construction_method',
 'occupancy_type',
 'manufactured_home_secured_property_type',
 'manufactured_home_land_property_interest',
 'total_units',
 'multifamily_affordable_units',
 'in

In [19]:
t = ipsc.table("tmp_df_table")

In [20]:
expr = t.count()

In [21]:
# expr

In [22]:
# ibis.show_sql(expr) <<--- this doesn't seem to work 

In [23]:
## Profiling

In [24]:
df_sample = df_rdd.sample(fraction=0.1, seed=42).cache()

In [25]:
df_sample.count()

22/09/26 22:43:07 WARN MemoryStore: Not enough space to cache rdd_33_0 in memory! (computed 268.9 MiB so far)
22/09/26 22:43:07 WARN BlockManager: Persisting block rdd_33_0 to disk instead.


22/09/26 22:46:04 WARN MemoryStore: Not enough space to cache rdd_33_0 in memory! (computed 268.9 MiB so far)


2568242

In [26]:
# df_rdd_cache = df_rdd.cache()

In [27]:
profile = spark_df_profiling.ProfileReport(df_sample)
profile.to_file(outputfile="../data/profile-report.html")

22/09/26 22:46:06 WARN MemoryStore: Not enough space to cache rdd_33_0 in memory! (computed 268.9 MiB so far)


22/09/26 22:46:08 WARN MemoryStore: Not enough space to cache rdd_33_0 in memory! (computed 268.9 MiB so far)


22/09/26 22:46:10 WARN MemoryStore: Not enough space to cache rdd_33_0 in memory! (computed 268.9 MiB so far)


22/09/26 22:46:13 WARN MemoryStore: Not enough space to cache rdd_33_0 in memory! (computed 268.9 MiB so far)


/Users/mjboothaus/code/github/mjboothaus/sparky/.venv_dev_trial-spark-local-pandas-api/lib/python3.9/site-packages/spark_df_profiling/base.py:408: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(describe_constant_1d(df, column))


...

/Users/mjboothaus/code/github/mjboothaus/sparky/.venv_dev_trial-spark-local-pandas-api/lib/python3.9/site-packages/spark_df_profiling/base.py:418: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(describe_categorical_1d(df, column))


22/09/26 23:06:52 WARN MemoryStore: Not enough space to cache rdd_33_0 in memory! (computed 268.9 MiB so far)


22/09/26 23:06:54 WARN MemoryStore: Not enough space to cache rdd_33_0 in memory! (computed 268.9 MiB so far)


22/09/26 23:06:58 WARN MemoryStore: Not enough space to cache rdd_33_0 in memory! (computed 268.9 MiB so far)


/Users/mjboothaus/code/github/mjboothaus/sparky/.venv_dev_trial-spark-local-pandas-api/lib/python3.9/site-packages/spark_df_profiling/base.py:418: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(describe_categorical_1d(df, column))
